In [2]:
!pip install pydataset

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 26.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pydataset: filename=pydataset-0.2.0-py3-none-any.whl size=15939415 sha256=f7ddb462de69bdef00db20a03b720ee3d555e066bbceda86178ca44ce47e098f
  Stored in directory: /root/.cache/pip/wheels/29/93/3f/af54c413cecaac292940342c61882d2a8848674175d0bb0889
Successfully built pydataset


In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error as MSE
from numpy import linalg as LA
from matplotlib.ticker import ScalarFormatter
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder, MinMaxScaler, RobustScaler
from sklearn.impute import KNNImputer
import seaborn as sns
from pydataset import data
from sklearn.datasets import fetch_california_housing, load_diabetes, fetch_openml
from statsmodels.datasets import get_rdataset,co2, sunspots, elnino, macrodata, nile, randhie
from pandas_datareader import get_data_fred
from sklearn.datasets import fetch_openml

# DVAW

In [4]:
class DVAW:
    def __init__(self, n_components=50,
                 dict_of_kernels={'Gaussian': 51, 'Laplacian': 25}):
        # Initialize DVAW model parameters
        self.n_components = n_components
        self.lambda_reg = 1 # Regularization parameter
        self.dict_of_kernels = dict_of_kernels
        # Generate kernel parameters based on dictionary
        self.gamma, self.kernel_list, self.num_rbf, self.num_lap, self.P = self._generate_kernel_params(dict_of_kernels)
        self.ran_feature = None # Stores random features for kernels
        self.experts_weights = None # Stores weights for individual kernel experts
        self.weights_vaw2 = None # Stores weights for the second layer (VAW2 combiner)

    def _generate_kernel_params(self, dict_of_kernels):
        # Generates gamma values and kernel types for each expert
        gamma = []
        kernel_list = []
        num_rbf, num_lap, = 0, 0
        P = 0 # Total number of kernels/experts
        for kernel, num_kernel in dict_of_kernels.items():
            P += num_kernel # Accumulate total number of experts
            if num_kernel > 0:
                gamma_values = np.logspace(-2, 2, num_kernel) # Generate gamma values for each kernel instance
                if kernel == 'Gaussian':
                    num_rbf = num_kernel
                    gamma.extend(gamma_values)
                    kernel_list.extend(['Gaussian'] * num_kernel)
                elif kernel == 'Laplacian':
                    num_lap = num_kernel
                    gamma.extend(gamma_values)
                    kernel_list.extend(['Laplacian'] * num_kernel)
                else:
                    raise ValueError(f"Unknown kernel type: {kernel}")
        return np.array(gamma), kernel_list, num_rbf, num_lap, P

    def generate_random_features_and_dict(self, X):
        # Generates random Fourier features for each kernel type
        np.random.seed(1) # For reproducibility
        M, N = X.shape # M samples, N original features
        self.ran_feature = np.zeros((N, self.n_components, self.P)) # Stores random projections for each kernel
        random_features = {} # Dictionary to hold generated features for each expert
        kernel_counters = {'Gaussian': 0, 'Laplacian': 0} # Counters for naming experts
        for i, kernel_type in enumerate(self.kernel_list):
            features = np.zeros((M, self.n_components * 2)) # Placeholder for current kernel's features
            if kernel_type == 'Gaussian':
                gamma = self.gamma[i]
                self.ran_feature[:, :, i] = np.random.randn(N, self.n_components) * np.sqrt(1 / gamma) # Random projections for Gaussian
                for j in range(M):
                    X_f = X[j:j + 1, :].dot(self.ran_feature[:, :, i])
                    features[j, :] = (1 / np.sqrt(self.n_components)) * np.concatenate((np.sin(X_f), np.cos(X_f)), axis=1) # Apply sin/cos for Fourier features
                random_features[f"{kernel_type}_{kernel_counters['Gaussian']}"] = features
                kernel_counters['Gaussian'] += 1
            elif kernel_type == 'Laplacian':
                gamma = self.gamma[i]
                laplacian_index = i - self.num_rbf # Adjust index for Laplacian kernels
                if 0 <= laplacian_index < self.num_lap:
                    self.ran_feature[:, :, i] = np.random.standard_cauchy((N, self.n_components)) * (1 / self.gamma[i]) # Random projections for Laplacian
                    for j in range(M):
                        X_f = X[j:j + 1, :].dot(self.ran_feature[:, :, i])
                        features[j, :] = (1 / np.sqrt(self.n_components)) * np.concatenate((np.sin(X_f), np.cos(X_f)), axis=1) # Apply sin/cos for Fourier features
                    random_features[f"{kernel_type}_{kernel_counters['Laplacian']}"] = features
                    kernel_counters['Laplacian'] += 1
                else:
                    raise IndexError("Laplacian index out of bounds")
            else:
                raise ValueError(f"Unknown kernel type: {kernel_type}")
        return random_features

    def _vaw_forecaster(self, features, target):
        # Implements the Vovk-Azoury-Warmuth (VAW) online linear regressor
        n_samples, n_features = features.shape
        predictions = []
        weights = np.zeros(n_features) # Final weights of the online algorithm
        reg_matrix_inverse = (1 / self.lambda_reg) * np.eye(n_features) # Initialize inverse regularization matrix
        sum_yizi = np.zeros(n_features) # Sum of target * feature vector for weight update
        for t in range(n_samples):
            zt = features[t] # Current feature vector
            yt = target[t] # Current target value
            if t == 0:
                prediction = 0 # Initial prediction for the first sample (can be customized)
            else:
                prediction = np.dot(sum_yizi, reg_matrix_inverse @ zt) # Calculate prediction based on current weights
            predictions.append(prediction)
            sum_yizi += yt * zt # Accumulate y_t * z_t
            zt = zt.reshape(-1, 1) # Reshape z_t for matrix operations
            # Update inverse regularization matrix using Sherman-Morrison formula
            numerator = reg_matrix_inverse @ zt @ zt.T @ reg_matrix_inverse
            denominator = 1 + zt.T @ reg_matrix_inverse @ zt
            reg_matrix_inverse = reg_matrix_inverse - (numerator / denominator[0, 0])
        if n_samples > 0:
            weights = reg_matrix_inverse @ sum_yizi # Compute final weights
        return np.array(predictions), weights

    def fit(self, X, Y):
        # Fits the DVAW model by training individual experts and then a second-layer VAW combiner
        self.fourier_features = self.generate_random_features_and_dict(X) # Generate random Fourier features for each expert
        self.experts_weights = np.zeros((self.P, self.n_components * 2)) # Initialize weights for each expert's linear model
        experts_predictions = pd.DataFrame() # DataFrame to store predictions from each expert

        i = 0
        for kernel_name, features in self.fourier_features.items():
            # Train each expert (using VAW forecaster) and get its predictions and weights
            predictions, self.experts_weights[i] = self._vaw_forecaster(features, Y)
            experts_predictions = pd.concat([experts_predictions,
                                             pd.DataFrame(predictions, columns=[kernel_name])], axis=1)
            i += 1
        # Train the second-layer VAW forecaster using expert predictions as input features
        self.predictions_vaw2, self.weights_vaw2 = self._vaw_forecaster(np.array(experts_predictions), Y)

    def predict(self, X):
        # Makes predictions on new data using the trained DVAW model
        M, N = X.shape # M samples, N original features
        random_features_list = [] # List to store generated random features for prediction
        experts_predictions = np.zeros((self.P, M)) # Array to store predictions from each expert

        # Generate random Fourier features for the new X data
        for i, kernel_type in enumerate(self.kernel_list):
            features = np.zeros((M, self.n_components * 2))
            for j in range(M):
                # Apply the same random projections used during training
                X_f = X[j:j + 1, :].dot(self.ran_feature[:, :, i])
                features[j, :] = (1 / np.sqrt(self.n_components)) * np.concatenate((np.sin(X_f), np.cos(X_f)), axis=1)
            random_features_list.append(features)
        random_features_array = np.array(random_features_list)

        # Calculate predictions from each expert using their learned weights
        for p in range(self.P):
            experts_predictions[p] = np.dot(random_features_array[p], self.experts_weights[p])

        # Combine expert predictions using the second-layer VAW weights
        vaw2_predictions = experts_predictions.T @ self.weights_vaw2
        return vaw2_predictions

# DVAW scaled

In [5]:
class DVAW_scaled:
    def __init__(self, n_components=50,
                 dict_of_kernels={'Gaussian': 51, 'Laplacian': 25},
                 x_scaler=None):
        """
        Initializes the DVAW_scaled model.

        Args:
            n_components (int): Number of random features for each kernel.
            dict_of_kernels (dict): Dictionary specifying kernel types and their counts.
                                    E.g., {'Gaussian': 51, 'Laplacian': 25}.
            x_scaler (sklearn.preprocessing.Scaler, optional): Scaler for input features X.
                                                               Defaults to StandardScaler if None.
        """
        self.n_components = n_components
        self.lambda_reg = 1 # Regularization parameter for VAW forecaster
        self.dict_of_kernels = dict_of_kernels

        # Initialize X scaler, using provided one or default StandardScaler
        self.x_scaler = x_scaler

        # Generate kernel parameters (gamma values, kernel list, counts)
        self.gamma, self.kernel_list, self.num_rbf, self.num_lap, self.P = self._generate_kernel_params(dict_of_kernels)

        self.ran_feature = None # Stores random projections for each kernel (set during fit)
        self.experts_weights = None # Stores learned weights for individual kernel experts
        self.weights_vaw2 = None # Stores learned weights for the second-layer VAW combiner

        np.random.seed(1) # Set random seed for reproducibility

    def _generate_kernel_params(self, dict_of_kernels):
        """
        Generates kernel parameters (gamma values and a list of kernel types)
        based on the provided dictionary.

        Args:
            dict_of_kernels (dict): Dictionary specifying kernel types and their counts.

        Returns:
            tuple: (gamma_array, kernel_type_list, num_gaussian_kernels, num_laplacian_kernels, total_kernels)
        """
        gamma = []
        kernel_list = []
        num_rbf, num_lap = 0, 0
        P = 0 # Total number of individual kernel experts
        for kernel, num_kernel in dict_of_kernels.items():
            P += num_kernel
            if num_kernel > 0:
                # Generate gamma values logarithmically spaced
                gamma_values = np.logspace(-2, 2, num_kernel)
                if kernel == 'Gaussian':
                    num_rbf = num_kernel
                    gamma.extend(gamma_values)
                    kernel_list.extend(['Gaussian'] * num_kernel)
                elif kernel == 'Laplacian':
                    num_lap = num_kernel
                    gamma.extend(gamma_values)
                    kernel_list.extend(['Laplacian'] * num_kernel)
                else:
                    raise ValueError(f"Unknown kernel type: {kernel}")
        return np.array(gamma), kernel_list, num_rbf, num_lap, P

    def generate_random_features_and_dict(self, X):
        """
        Generates random projection matrices (ran_feature_matrix) and
        corresponding Fourier features (random_features) for the given input X.

        Args:
            X (np.array): Input features (expected to be already scaled).

        Returns:
            tuple: (random_features_dict, random_feature_matrix)
                random_features_dict (dict): Dictionary of Fourier features for each kernel.
                random_feature_matrix (np.array): Matrix of random projections for each kernel.
        """
        M, N = X.shape # M: number of samples, N: number of original features

        # ran_feature_matrix is local and will be returned and stored in self.ran_feature
        ran_feature_matrix = np.zeros((N, self.n_components, self.P))
        random_features = {} # Dictionary to store Fourier features for each expert
        kernel_counters = {'Gaussian': 0, 'Laplacian': 0} # Counters for naming experts

        for i, kernel_type in enumerate(self.kernel_list):
            features = np.zeros((M, self.n_components * 2)) # Fourier features for the current kernel
            gamma = self.gamma[i] # Gamma value for the current kernel

            if kernel_type == 'Gaussian':
                ran_feature_matrix[:, :, i] = np.random.randn(N, self.n_components) * np.sqrt(1 / gamma)
            elif kernel_type == 'Laplacian':
                laplacian_index = i - self.num_rbf
                if not (0 <= laplacian_index < self.num_lap):
                    raise IndexError("Laplacian index out of bounds")
                ran_feature_matrix[:, :, i] = np.random.standard_cauchy((N, self.n_components)) * (1 / gamma)
            else:
                raise ValueError(f"Unknown kernel type: {kernel_type}")

            # Compute Fourier features for the current kernel
            for j in range(M):
                X_f = X[j:j + 1, :].dot(ran_feature_matrix[:, :, i])
                features[j, :] = (1 / np.sqrt(self.n_components)) * np.concatenate((np.sin(X_f), np.cos(X_f)), axis=1)

            # Add Fourier features to the dictionary
            random_features[f"{kernel_type}_{kernel_counters[kernel_type]}"] = features
            kernel_counters[kernel_type] += 1

        return random_features, ran_feature_matrix

    def _vaw_forecaster(self, features, target):
        """
        Implements the Vovk-Azoury-Warmuth (VAW) online linear regressor.

        Args:
            features (np.array): Feature matrix (n_samples, n_features).
            target (np.array): Target values vector (n_samples,).

        Returns:
            tuple: (predictions, weights)
                predictions (np.array): Vector of online predictions.
                weights (np.array): Learned weights.
        """
        n_samples, n_features = features.shape
        predictions = []
        weights = np.zeros(n_features) # Final weights of the online algorithm
        reg_matrix_inverse = (1 / self.lambda_reg) * np.eye(n_features) # Initialize inverse regularization matrix
        sum_yizi = np.zeros(n_features) # Sum of target * feature vector for weight update

        for t in range(n_samples):
            zt = features[t] # Current feature vector
            yt = target[t] # Current target value

            if t == 0:
                prediction = 0 # Initial prediction for the first sample
            else:
                prediction = np.dot(sum_yizi, reg_matrix_inverse @ zt) # Calculate prediction based on current weights
            predictions.append(prediction)

            sum_yizi += yt * zt # Accumulate y_t * z_t
            zt_reshaped = zt.reshape(-1, 1) # Reshape z_t for matrix operations

            # Update inverse regularization matrix using Sherman-Morrison formula
            numerator = reg_matrix_inverse @ zt_reshaped @ zt_reshaped.T @ reg_matrix_inverse
            denominator = 1 + zt_reshaped.T @ reg_matrix_inverse @ zt_reshaped

            # Check for division by zero, though unlikely with proper lambda_reg
            if denominator[0, 0] == 0:
                pass # Skipping update if denominator is zero
            else:
                reg_matrix_inverse = reg_matrix_inverse - (numerator / denominator[0, 0])

        if n_samples > 0:
            weights = reg_matrix_inverse @ sum_yizi # Compute final weights
        return np.array(predictions), weights

    def fit(self, X, Y):
        """
        Fits the DVAW_scaled model on input data X and Y.

        Args:
            X (np.array or pd.DataFrame): Input features.
            Y (np.array or pd.Series): Target values.
        """
        # Ensure Y is a numpy array if it's a pandas Series
        Y_np = Y.values if isinstance(Y, pd.Series) else Y

        # Define training size for scaler fitting (e.g., 75% of data)
        train_size = int(0.75 * X.shape[0])

        # 1. Scale input features X
        if self.x_scaler is not None:
          self.x_scaler.fit(X[:train_size]) # Fit scaler only on a portion of data
          X_scaled = self.x_scaler.transform(X) # Transform all data

        else:
          X_scaled = X

        # Generate random features and Fourier features based on scaled X
        self.fourier_features, self.ran_feature = self.generate_random_features_and_dict(X_scaled)

        # Initialize weights for experts
        self.experts_weights = np.zeros((self.P, self.n_components * 2))
        experts_predictions = pd.DataFrame() # DataFrame to store online predictions from each expert

        i = 0
        # First layer VAW: train individual experts for each kernel
        for kernel_name, features in self.fourier_features.items():
            # Train experts using original Y
            predictions, self.experts_weights[i] = self._vaw_forecaster(features, Y_np)
            experts_predictions = pd.concat([experts_predictions,
                                             pd.DataFrame(predictions, columns=[kernel_name])], axis=1)
            i += 1

        # Second layer VAW: combine expert predictions
        # Train the second layer using expert online predictions and original Y
        self.predictions_vaw2, self.weights_vaw2 = self._vaw_forecaster(np.array(experts_predictions), Y_np)

    def predict(self, X):
        """
        Makes predictions on new data X, including scaling of input features.

        Args:
            X (np.array or pd.DataFrame): Input features for prediction.

        Returns:
            np.array: Final model predictions, in the original Y range.
        """
        # 1. Scale new input features X using the fitted x_scaler
        if self.x_scaler is not None:
          X_scaled = self.x_scaler.transform(X)
        else: X_scaled = X

        M, N = X_scaled.shape
        random_features_list = [] # List to store generated random features for prediction

        # Initialize experts_predictions_from_features with correct shape: (num_samples, num_experts)
        experts_predictions_from_features = np.zeros((M, self.P))

        # Generate Fourier features for each kernel based on the scaled X data
        for i, kernel_type in enumerate(self.kernel_list):
            features = np.zeros((M, self.n_components * 2))
            for j in range(M):
                # Apply the same random projections (self.ran_feature) used during training
                X_f = X_scaled[j:j + 1, :].dot(self.ran_feature[:, :, i])
                features[j, :] = (1 / np.sqrt(self.n_components)) * np.concatenate((np.sin(X_f), np.cos(X_f)), axis=1)
            random_features_list.append(features)

        # Calculate predictions from each expert using their learned weights
        for p in range(self.P):
            experts_predictions_from_features[:, p] = np.dot(random_features_list[p], self.experts_weights[p])

        # Apply the second-layer VAW weights to combine expert predictions.
        # Predictions will be in the original Y range since Y was not scaled during fit.
        final_predictions = experts_predictions_from_features @ self.weights_vaw2

        return final_predictions

# TVAW

In [6]:
class TVAW:
    def __init__(self, n_components=50,
                 dict_of_kernels={'Gaussian': 51, 'Laplacian': 25}):
        self.n_components = n_components
        self.lambda_reg = 1
        self.dict_of_kernels = dict_of_kernels

        # Initialize kernel parameters
        self.gamma, self.kernel_list, self.num_rbf, self.num_lap, self.P = self._generate_kernel_params(dict_of_kernels)

        self.final_predictions = None
        self.final_weights = None

        # List of X-scaler options, including None (no scaling)
        self.x_scalers_options = [
            ('StandardScaler', StandardScaler),
            ('MinMaxScaler', MinMaxScaler),
            ('RobustScaler', RobustScaler),
            ('NoScaler', None)
        ]
        # Y-scaler class option
        self.y_scaler_option = MinMaxScaler

        # Stores trained scalers and model components for each combination
        self.scaler_combinations = []

        # Dictionary to store ALL trained scalers, random features, and weights for each combination
        self.trained_scalers = {}

        # Set random seed for reproducibility of random features
        np.random.seed(1)

        # Generate all possible X and Y scaler combinations
        self._generate_scaler_combinations()

    def _generate_kernel_params(self, dict_of_kernels):
        """
        Generates kernel parameters (gamma values and kernel types).
        """
        gamma = []
        kernel_list = []
        num_rbf, num_lap = 0, 0
        P = 0 # Total number of kernels
        for kernel, num_kernel in dict_of_kernels.items():
            P += num_kernel
            if num_kernel > 0:
                gamma_values = np.logspace(-2, 2, num_kernel)
                if kernel == 'Gaussian':
                    num_rbf = num_kernel
                    gamma.extend(gamma_values)
                    kernel_list.extend(['Gaussian'] * num_kernel)
                elif kernel == 'Laplacian':
                    num_lap = num_kernel
                    gamma.extend(gamma_values)
                    kernel_list.extend(['Laplacian'] * num_kernel)
                else:
                    raise ValueError(f"Unknown kernel type: {kernel}")
        return np.array(gamma), kernel_list, num_rbf, num_lap, P

    def _generate_scaler_combinations(self):
        """
        Creates a list of all X and Y scaler combinations.
        """
        y_scaler_templates = [
            ('Yraw', None), # No scaling for Y
            ('Yscaled', self.y_scaler_option) # MinMaxScaler for Y
        ]

        for x_name, x_scaler_class in self.x_scalers_options:
            for y_name, y_scaler_class in y_scaler_templates:
                # Create X-scaler instance or None
                current_x_scaler_instance = x_scaler_class() if x_scaler_class is not None else None
                # Create Y-scaler instance or None
                current_y_scaler_instance = y_scaler_class() if y_scaler_class is not None else None

                combo_full_name = f"{x_name}_{y_name}"
                self.scaler_combinations.append((combo_full_name, current_x_scaler_instance, current_y_scaler_instance))

    def generate_random_features_and_dict(self, X):
        """
        Generates random projection matrices and corresponding Fourier features for X.
        """
        M, N = X.shape
        ran_feature_matrix = np.zeros((N, self.n_components, self.P))
        random_features = {}
        kernel_counters = {'Gaussian': 0, 'Laplacian': 0}

        for i, kernel_type in enumerate(self.kernel_list):
            features = np.zeros((M, self.n_components * 2))
            gamma = self.gamma[i]

            if kernel_type == 'Gaussian':
                ran_feature_matrix[:, :, i] = np.random.randn(N, self.n_components) * np.sqrt(1 / gamma)
            elif kernel_type == 'Laplacian':
                laplacian_index = i - self.num_rbf
                if not (0 <= laplacian_index < self.num_lap):
                    raise IndexError("Laplacian index out of bounds")
                ran_feature_matrix[:, :, i] = np.random.standard_cauchy((N, self.n_components)) * (1 / gamma)
            else:
                raise ValueError(f"Unknown kernel type: {kernel_type}")

            # Compute Fourier features for the current kernel
            for j in range(M):
                X_f = X[j:j + 1, :].dot(ran_feature_matrix[:, :, i])
                features[j, :] = (1 / np.sqrt(self.n_components)) * np.concatenate((np.sin(X_f), np.cos(X_f)), axis=1)

            random_features[f"{kernel_type}_{kernel_counters[kernel_type]}"] = features
            kernel_counters[kernel_type] += 1

        return random_features, ran_feature_matrix

    def _vaw_forecaster(self, features, target):
        """
        Implements the Vovk-Azoury-Warmuth (VAW) online linear regressor.
        """
        n_samples, n_features = features.shape
        predictions = []

        reg_matrix_inverse = (1 / self.lambda_reg) * np.eye(n_features)
        sum_yizi = np.zeros(n_features)

        for t in range(n_samples):
            zt = features[t]
            yt = target[t]

            if t == 0:
                prediction = 0
            else:
                prediction = np.dot(sum_yizi, reg_matrix_inverse @ zt)
            predictions.append(prediction)

            sum_yizi += yt * zt

            zt_reshaped = zt.reshape(-1, 1)
            numerator = reg_matrix_inverse @ zt_reshaped @ zt_reshaped.T @ reg_matrix_inverse
            denominator = 1 + zt_reshaped.T @ reg_matrix_inverse @ zt_reshaped

            if denominator[0, 0] == 0:
                print("Warning: Denominator in VAW update is zero. Skipping update.")
            else:
                reg_matrix_inverse = reg_matrix_inverse - (numerator / denominator[0, 0])

        weights = np.zeros(n_features)
        if n_samples > 0:
            weights = reg_matrix_inverse @ sum_yizi

        return np.array(predictions), weights

    def fit(self, X, Y):
        """
        Fits the TVAW model on input data X and Y.
        """
        Y_original_np = Y.values if isinstance(Y, pd.Series) else Y
        Y_2d_original = Y_original_np.reshape(-1, 1)

        combined_predictions_for_final_layer = pd.DataFrame()
        train_size = int(X.shape[0]*0.75)
        # Iterate through each scaler combination (Level 2)
        for combo_name, x_scaler_instance, y_scaler_instance in self.scaler_combinations:
            # 1. Scale X for the current combination
            if x_scaler_instance is not None:
                x_scaler_instance.fit(X[:train_size])
                X_scaled = x_scaler_instance.transform(X)
                self.trained_scalers[f"{combo_name}_x_scaler"] = x_scaler_instance
            else:
                X_scaled = X.copy()
                self.trained_scalers[f"{combo_name}_x_scaler"] = None

            # 2. Scale Y for the current combination
            if y_scaler_instance is not None:
                y_scaler_instance.fit(Y_2d_original[:train_size])
                Y_scaled = y_scaler_instance.transform(Y_2d_original).flatten()
                self.trained_scalers[f"{combo_name}_y_scaler"] = y_scaler_instance
            else:
                Y_scaled = Y_original_np.copy()
                self.trained_scalers[f"{combo_name}_y_scaler"] = None

            # 3. Level 1 VAW: Generate random features and train experts
            fourier_features_for_combo, ran_feature_for_combo = self.generate_random_features_and_dict(X_scaled)
            self.trained_scalers[f"{combo_name}_ran_feature"] = ran_feature_for_combo

            experts_weights_for_combo = np.zeros((self.P, self.n_components * 2))
            experts_online_predictions_df = pd.DataFrame()

            i = 0
            for kernel_name, features in fourier_features_for_combo.items():
                preds_online, experts_weights_for_combo[i] = self._vaw_forecaster(features, Y_scaled)
                experts_online_predictions_df = pd.concat([
                    experts_online_predictions_df,
                    pd.DataFrame(preds_online, columns=[f"{combo_name}_{kernel_name}"])
                ], axis=1)
                i += 1
            self.trained_scalers[f"{combo_name}_experts_weights"] = experts_weights_for_combo

            # 4. Level 2 VAW: Combine expert predictions for the current combination
            level2_preds_scaled, weights_vaw2_for_combo = self._vaw_forecaster(np.array(experts_online_predictions_df), Y_scaled)
            self.trained_scalers[f"{combo_name}_weights_vaw2"] = weights_vaw2_for_combo

            # 5. Inverse scale Level 2 predictions if Y was scaled
            if y_scaler_instance is not None:
                level2_preds_unscaled = y_scaler_instance.inverse_transform(level2_preds_scaled.reshape(-1, 1)).flatten()
            else:
                level2_preds_unscaled = level2_preds_scaled.copy()

            combined_predictions_for_final_layer = pd.concat([
                combined_predictions_for_final_layer,
                pd.DataFrame(level2_preds_unscaled, columns=[combo_name])
            ], axis=1)

        # 6. Level 3 VAW: Final combination of all 8 predictions
        self.final_predictions, self.final_weights = self._vaw_forecaster(
            np.array(combined_predictions_for_final_layer), Y_original_np
        )

    def predict(self, X):
        """
        Makes predictions on new data X.
        """
        combined_predictions_for_final_layer_predict = pd.DataFrame()

        # Iterate through each scaler combination, using stored trained objects
        for combo_name, _, _ in self.scaler_combinations:
            # Retrieve trained scalers and components for the current combination
            x_scaler_trained = self.trained_scalers.get(f"{combo_name}_x_scaler")
            y_scaler_trained = self.trained_scalers.get(f"{combo_name}_y_scaler")
            ran_feature_for_combo = self.trained_scalers.get(f"{combo_name}_ran_feature")
            experts_weights_for_combo = self.trained_scalers.get(f"{combo_name}_experts_weights")
            weights_vaw2_for_combo = self.trained_scalers.get(f"{combo_name}_weights_vaw2")

            if any(item is None for item in [ran_feature_for_combo, experts_weights_for_combo, weights_vaw2_for_combo]):
                # If basic components are missing, this combination cannot predict correctly.
                combined_predictions_for_final_layer_predict = pd.concat([
                    combined_predictions_for_final_layer_predict,
                    pd.DataFrame(np.zeros(len(X)), columns=[combo_name])
                ], axis=1)
                continue

            # 1. Scale X for prediction
            if x_scaler_trained is not None:
                X_scaled = x_scaler_trained.transform(X)
            else:
                X_scaled = X.copy()

            M, N = X_scaled.shape
            experts_predictions_from_features = np.zeros((M, self.P))
            random_features_list = []

            # Generate Fourier features for each kernel
            for i, kernel_type in enumerate(self.kernel_list):
                features = np.zeros((M, self.n_components * 2))
                for j in range(M):
                    X_f = X_scaled[j:j + 1, :].dot(ran_feature_for_combo[:, :, i])
                    features[j, :] = (1 / np.sqrt(self.n_components)) * np.concatenate(
                        (np.sin(X_f), np.cos(X_f)), axis=1
                    )
                random_features_list.append(features)

            # Calculate predictions from each expert
            for p in range(self.P):
                experts_predictions_from_features[:, p] = np.dot(random_features_list[p], experts_weights_for_combo[p])

            # 2. Apply Level 2 VAW weights. Predictions will be in the Y_scaled scale (if Y was scaled)
            level2_preds_scaled = experts_predictions_from_features @ weights_vaw2_for_combo

            # 3. Inverse scale Level 2 predictions if Y was scaled
            if y_scaler_trained is not None:
                level2_preds_unscaled = y_scaler_trained.inverse_transform(level2_preds_scaled.reshape(-1, 1)).flatten()
            else:
                level2_preds_unscaled = level2_preds_scaled.copy()

            combined_predictions_for_final_layer_predict = pd.concat([
                combined_predictions_for_final_layer_predict,
                pd.DataFrame(np.zeros(len(X)) if level2_preds_unscaled is None else level2_preds_unscaled, columns=[combo_name])
            ], axis=1)


        # 4. Final prediction - combine all 8 predictions using final_weights
        final_pred = combined_predictions_for_final_layer_predict.values @ self.final_weights

        return final_pred

# Preprocessing of all used datasets in the tests (uncomment the nessesary)

In [9]:
#---------Airfoil--------
'''
df = pd.DataFrame(np.genfromtxt('airfoil_self_noise.dat'))
target = 5
'''

# -------Concrete--------
'''
df = pd.read_excel('Concrete_Data.xls')
target = 'Concrete compressive strength(MPa, megapascals) '
'''

#--------Bias-----------
'''
data = np.genfromtxt('Bias_correction_ucl.csv', skip_header=1, delimiter=',')
imputer = KNNImputer(n_neighbors=5)
imputer = imputer.fit(data)
df = pd.DataFrame(imputer.transform(data))
df.drop(22, axis = 1, inplace=True)
target = 23
'''

#----------Naval------------
'''
data = np.genfromtxt('NavalData.txt')
df = pd.DataFrame(data)
target = 0
'''

# ----------Mercedes-Benz Greener Manufacturing--------------
'''
df = pd.read_csv('train.csv')
df = df.drop('ID', axis = 1)
target = 'y'

categorical_cols = [col for col in df.columns if df[col].dtype == 'object']
df = pd.get_dummies(df, columns=categorical_cols, prefix=categorical_cols, dtype = int)
'''

#-----------house-prices-advanced-regression-techniques-------------------
'''
df = pd.read_csv('train.csv')
df.drop('Id', axis = 1, inplace = True)
target = 'SalePrice'
float_cols = [col for col in df.columns if df[col].dtype == float]

imputer = KNNImputer(n_neighbors=5)
df[float_cols] = imputer.fit_transform(df[float_cols])

categorical_cols = [col for col in df.columns if df[col].dtype == 'object']
for col in categorical_cols:
  df[col] = df[col].fillna('NaN')
df = pd.get_dummies(df, columns=categorical_cols, prefix=categorical_cols, dtype = int)
'''


#-----------------Behavior of the urban traffic of the city of Sao Paulo in Brazil------------------------
'''
df = pd.read_csv('Behavior of the urban traffic of the city of Sao Paulo in Brazil.csv', sep =';')
target = df.columns[-1]
df[target] = df[target].astype('string').apply(lambda x: float(x.replace(',', '.')))
'''

#---------Diamonds-----------
'''
df = sns.load_dataset('diamonds').sample(500, random_state=42)
target = 'price'
cat_cols = ['cut', 'color', 'clarity']
df = pd.get_dummies(df, columns=cat_cols, prefix=cat_cols, dtype = int)
'''

#---------Tips-----------

df = sns.load_dataset('tips')
target = 'tip'

categorical_features = ['sex', 'smoker', 'day', 'time']

encoder = OneHotEncoder(drop='first', sparse_output=False)
X_cat_encoded = encoder.fit_transform(df[categorical_features])

encoded_cat_columns = encoder.get_feature_names_out(categorical_features)
X_cat_encoded_df = pd.DataFrame(X_cat_encoded, columns=encoded_cat_columns)

numeric_features = ['total_bill', 'size', 'tip']
df = pd.concat([df[numeric_features], X_cat_encoded_df], axis=1)

#---------Boston-----------
'''
target = 'medv'
df = data('Boston')
'''
#---------California-----------
'''
target = 'MedHouseVal'
df = fetch_california_housing(as_frame=True).frame.iloc[:10000]
'''

#---------Mtcars-----------
'''
target = 'mpg'
df = get_rdataset('mtcars').data
'''

#---------Energy Efficiency-----------
'''
data = fetch_openml(name='energy_efficiency', as_frame=True, parser='auto')
target = 'Y1'
df = data.frame
'''


"\ndata = fetch_openml(name='energy_efficiency', as_frame=True, parser='auto')\ntarget = 'Y1'\ndf = data.frame\n"

# Running the DVAW model (usage is similar for DVAW_scaled and TVAW)

In [11]:
#Initializating data
X = df.drop(target, axis = 1).values
Y = df[target].values

train_size = int(df.shape[0]*0.75)

# Split data into training and testing sets
X_test = X[train_size:]
Y_test = Y[train_size:]

# Train the model
predictor = DVAW()
predictor.fit(X, Y)

#Make predictions on test
preds = predictor.predict(X_test)

# Print MSE
print('MSE is ', MSE(preds, Y_test))

MSE is  1.1927428954550534
